# 5. Model Degerlendirme

Bu notebook feature importance, SHAP analizi ve business uyumlulugu kontrolu icerir.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

RANDOM_STATE = 42

## 1. Feature Importance

In [ ]:
# importance = pd.DataFrame({
#     'feature': X.columns,
#     'importance': model.feature_importances_
# }).sort_values('importance', ascending=False)

## 2. SHAP Analizi

In [ ]:
# explainer = shap.TreeExplainer(model)
# shap_values = explainer.shap_values(X)
# shap.summary_plot(shap_values, X)

## 3. Business Uyumlulugu

In [ ]:
# Esik degeri analizi, precision-recall trade-off

## 4. Sonuclar

### Baseline vs Final
| Metrik | Baseline | Final | Gelisim |
|--------|----------|-------|--------|
| AUC | X.XXXX | X.XXXX | +X.XX% |